In [91]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklift.metrics import uplift_at_k, qini_auc_score
from sklift.models import SoloModel
from catboost import CatBoostClassifier
from sklift.models import ClassTransformation

In [92]:
clients = pd.read_csv('uplift-focus/x5-uplift-valid/data/clients2.csv').set_index('client_id')
products = pd.read_csv('uplift-focus/x5-uplift-valid/data/products.csv')
train = pd.read_csv('uplift-focus/x5-uplift-valid/data/train.csv').set_index('client_id')
test = pd.read_csv('uplift-focus/x5-uplift-valid/data/test.csv')


In [93]:
train_purch = pd.read_csv('uplift-focus/x5-uplift-valid/train_purch/train_purch.csv')
test_purch = pd.read_csv('uplift-focus/x5-uplift-valid/test_purch/test_purch.csv')

In [94]:
test_purch

,client_id,transaction_id,transaction_datetime,regular_points_received,express_points_received,regular_points_spent,express_points_spent,purchase_sum,store_id,product_id,product_quantity,trn_sum_from_iss,trn_sum_from_red
0,00010925a5,e91ffe5b50,2018-11-25 10:56:18,1.5,0.0,0.0,0.0,303.0,76d287ce03,a375a035b9,2.0,50.0,NaN
1,00010925a5,e91ffe5b50,2018-11-25 10:56:18,1.5,0.0,0.0,0.0,303.0,76d287ce03,6f9e02ee36,1.0,90.0,NaN
2,00010925a5,e91ffe5b50,2018-11-25 10:56:18,1.5,0.0,0.0,0.0,303.0,76d287ce03,42e1eb8d23,1.0,20.0,NaN
3,00010925a5,e91ffe5b50,2018-11-25 10:56:18,1.5,0.0,0.0,0.0,303.0,76d287ce03,5c6de3010f,2.0,85.0,NaN
4,00010925a5,e91ffe5b50,2018-11-25 10:56:18,1.5,0.0,0.0,0.0,303.0,76d287ce03,343e841aaa,1.0,58.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6883733,fffff6ce77,48f6d115ce,2019-03-15 17:47:02,6.7,0.0,0.0,0.0,676.0,5ec64bdccb,24d5a54f42,1.0,164.0,NaN
6883734,fffff6ce77,48f6d115ce,2019-03-15 17:47:02,6.7,0.0,0.0,0.0,676.0,5ec64bdccb,7427d0830d,1.0,134.0,NaN
6883735,fffff6ce77,48f6d115ce,2019-03-15 17:47:02,6.7,0.0,0.0,0.0,676.0,5ec64bdccb,6c547c2ca8,1.0,60.0,NaN
6883736,fffff6ce77,48f6d115ce,2019-03-15 17:47:02,6.7,0.0,0.0,0.0,676.0,5ec64bdccb,d04e7a8338,1.0,48.0,NaN


In [95]:
clients.head()

,client_id.1,first_issue_date,first_redeem_date,age,gender
client_id,,,,,
000012768d,NaN,2017-08-05 15:40:48,2018-01-04 19:30:07,45,U
000036f903,NaN,2017-04-10 13:54:23,2017-04-23 12:37:56,72,F
00010925a5,NaN,2018-07-24 16:21:29,2018-09-14 16:12:49,83,U
0001f552b0,NaN,2017-06-30 19:20:38,2018-08-28 12:59:45,33,F
00020e7b18,NaN,2017-11-27 11:41:45,2018-01-10 17:50:05,73,U


In [96]:
clients.drop('client_id.1', axis = 1, inplace = True)

In [97]:
clients.describe()

,age
count,200039.000000
mean,46.417329
std,49.532475
min,-7491.000000
25%,34.000000
50%,45.000000
75%,59.000000
max,1852.000000


In [98]:
q_upper = np.quantile(clients['age'],0.99)
q_lower = np.quantile(clients['age'],0.01)

clients_age_cleaned = clients[(clients['age'] < q_upper) & (clients['age'] > q_lower)]['age']
clients_age_cleaned

client_id
000012768d    45
000036f903    72
0001f552b0    33
00020e7b18    73
00022fd34f    65
              ..
fffe0abb97    35
fffe0ed719    69
fffea1204c    73
fffeca6d22    77
fffff6ce77    42
Name: age, Length: 195110, dtype: int64

In [99]:
clients

,first_issue_date,first_redeem_date,age,gender
client_id,,,,
000012768d,2017-08-05 15:40:48,2018-01-04 19:30:07,45,U
000036f903,2017-04-10 13:54:23,2017-04-23 12:37:56,72,F
00010925a5,2018-07-24 16:21:29,2018-09-14 16:12:49,83,U
0001f552b0,2017-06-30 19:20:38,2018-08-28 12:59:45,33,F
00020e7b18,2017-11-27 11:41:45,2018-01-10 17:50:05,73,U
...,...,...,...,...
fffe0abb97,2017-11-27 08:56:54,2018-02-11 09:26:08,35,F
fffe0ed719,2017-09-15 08:53:24,2017-12-12 14:50:12,69,U
fffea1204c,2018-01-31 16:59:37,2018-03-12 17:02:27,73,F


In [100]:
clients['first_issue_date'] = pd.to_datetime(clients['first_issue_date'])
clients['first_issue_date_d'] = clients['first_issue_date'].dt.date
clients_issues_dates = pd.DataFrame(clients.groupby('first_issue_date_d').size()).\
reset_index().rename(columns = {0: 'Number of clients'})
clients

,first_issue_date,first_redeem_date,age,gender,first_issue_date_d
client_id,,,,,
000012768d,2017-08-05 15:40:48,2018-01-04 19:30:07,45,U,2017-08-05
000036f903,2017-04-10 13:54:23,2017-04-23 12:37:56,72,F,2017-04-10
00010925a5,2018-07-24 16:21:29,2018-09-14 16:12:49,83,U,2018-07-24
0001f552b0,2017-06-30 19:20:38,2018-08-28 12:59:45,33,F,2017-06-30
00020e7b18,2017-11-27 11:41:45,2018-01-10 17:50:05,73,U,2017-11-27
...,...,...,...,...,...
fffe0abb97,2017-11-27 08:56:54,2018-02-11 09:26:08,35,F,2017-11-27
fffe0ed719,2017-09-15 08:53:24,2017-12-12 14:50:12,69,U,2017-09-15
fffea1204c,2018-01-31 16:59:37,2018-03-12 17:02:27,73,F,2018-01-31


In [101]:
train_purch.head()

,client_id,transaction_id,transaction_datetime,regular_points_received,express_points_received,regular_points_spent,express_points_spent,purchase_sum,store_id,product_id,product_quantity,trn_sum_from_iss,trn_sum_from_red
0,000012768d,7e3e2e3984,2018-12-01 07:12:45,10.0,0.0,0.0,0.0,1007.0,54a4a11a29,9a80204f78,2.0,80.0,NaN
1,000012768d,7e3e2e3984,2018-12-01 07:12:45,10.0,0.0,0.0,0.0,1007.0,54a4a11a29,da89ebd374,1.0,65.0,NaN
2,000012768d,7e3e2e3984,2018-12-01 07:12:45,10.0,0.0,0.0,0.0,1007.0,54a4a11a29,0a95e1151d,1.0,24.0,NaN
3,000012768d,7e3e2e3984,2018-12-01 07:12:45,10.0,0.0,0.0,0.0,1007.0,54a4a11a29,4055b15e4a,2.0,50.0,NaN
4,000012768d,7e3e2e3984,2018-12-01 07:12:45,10.0,0.0,0.0,0.0,1007.0,54a4a11a29,a685f1916b,1.0,22.0,NaN


In [102]:
clients['first_issue_date'] = pd.to_datetime(clients['first_issue_date'])
clients['first_issue_date_d'] = clients['first_issue_date'].dt.date
clients_issues_dates = pd.DataFrame(clients.groupby('first_issue_date_d').size()).\
reset_index().rename(columns = {0: 'Number of clients'})

In [103]:
train

,treatment_flg,purchased
client_id,,
ad6561e2d8,1,1
7c1ccbf93f,1,1
b58fadcab6,1,1
e99e6fabb9,0,0
27fb6f8520,1,1
...,...,...
999d284453,1,1
f634deea4e,0,1
16cb4f99b0,0,1


In [104]:
train_df = clients.merge(train,how = 'right', on = 'client_id')
test_df = clients.merge(test, how  = 'right', on = 'client_id')
train_df

,first_issue_date,first_redeem_date,age,gender,first_issue_date_d,treatment_flg,purchased
client_id,,,,,,,
ad6561e2d8,2017-12-03 17:33:51,2018-05-23 18:59:01,50,F,2017-12-03,1,1
7c1ccbf93f,2017-11-10 16:33:49,2018-02-22 19:02:25,24,F,2017-11-10,1,1
b58fadcab6,2017-11-02 21:17:45,2018-09-15 21:02:16,36,U,2017-11-02,1,1
e99e6fabb9,2018-05-16 10:21:20,2018-05-30 20:49:24,79,F,2018-05-16,0,0
27fb6f8520,2017-07-17 19:36:28,2017-12-15 10:05:25,34,F,2017-07-17,1,1
...,...,...,...,...,...,...,...
999d284453,2018-10-31 13:10:37,2019-04-06 13:49:41,72,U,2018-10-31,1,1
f634deea4e,2018-02-04 13:51:32,2018-03-12 17:41:42,35,U,2018-02-04,0,1
16cb4f99b0,2018-02-23 10:02:33,2019-05-31 17:37:36,50,F,2018-02-23,0,1


In [105]:
test_df

,client_id,first_issue_date,first_redeem_date,age,gender,first_issue_date_d
0,a9a604ed6e,2018-09-13 17:36:41,NaN,36,F,2018-09-13
1,ebd7360016,2017-07-03 17:12:12,2017-09-01 16:15:12,63,F,2017-07-03
2,908cd9b8e8,2018-03-30 17:01:20,2018-07-13 17:05:52,49,F,2018-03-30
3,dceb8ce861,2017-10-06 17:06:42,2018-08-11 18:43:58,46,U,2017-10-06
4,f4f0ac6b06,2018-06-27 21:39:41,2019-02-15 20:14:37,119,U,2018-06-27
...,...,...,...,...,...,...
60007,4762fb04c5,2018-05-31 16:03:28,2018-12-26 14:03:30,26,M,2018-05-31
60008,533d1516e7,2017-09-29 08:27:16,2018-01-07 17:36:31,37,M,2017-09-29
60009,c93b21a707,2018-10-03 21:02:26,2019-03-30 17:40:42,42,U,2018-10-03
60010,3fe0668de5,2018-12-25 18:27:14,2019-02-02 10:30:53,33,U,2018-12-25


In [106]:
train_df['first_issue_time'] = \
    (pd.to_datetime(train_df['first_issue_date'])
     - pd.Timestamp('1970-01-01')) // pd.Timedelta('1s')


test_df['first_issue_time'] = \
    (pd.to_datetime(test_df['first_issue_date'])
     - pd.Timestamp('1970-01-01')) // pd.Timedelta('1s')


train_df['first_redeem_time'] = \
    (pd.to_datetime(train_df['first_redeem_date'])
     - pd.Timestamp('1970-01-01')) // pd.Timedelta('1s')


test_df['first_redeem_time'] = \
    (pd.to_datetime(test_df['first_redeem_date'])
     - pd.Timestamp('1970-01-01')) // pd.Timedelta('1s')


train_df['issue_redeem_delay'] = train_df['first_redeem_time'] \
    - train_df['first_issue_time']


test_df['issue_redeem_delay'] = test_df['first_redeem_time'] \
    - test_df['first_issue_time']


In [107]:
purchases_train = train_purch.merge(train,how = 'right', on = 'client_id')
purchases_features = purchases_train.groupby('client_id')[['transaction_id','product_id']].nunique().reset_index()

purchases_test = test_purch.merge(test_df,how = 'right', on = 'client_id')
purchases_test_features = purchases_test.groupby('client_id')[['transaction_id','product_id']].nunique().reset_index()
purchases_test_features

,client_id,transaction_id,product_id
0,00010925a5,18,58
1,00035a21d9,3,25
2,00038f9200,48,100
3,0004315e57,14,93
4,0006fca4bf,3,28
...,...,...,...
60007,fff9aa288c,10,51
60008,fff9f772f6,4,77
60009,fffe0ed719,30,89
60010,fffea1204c,17,45


In [108]:
purchases_train

,client_id,transaction_id,transaction_datetime,regular_points_received,express_points_received,regular_points_spent,express_points_spent,purchase_sum,store_id,product_id,product_quantity,trn_sum_from_iss,trn_sum_from_red,treatment_flg,purchased
0,ad6561e2d8,c4cd2780ab,2018-11-23 13:42:50,5.9,0.0,0.0,0.0,593.92,658cf09cd4,1bc3da0e85,1.0,40.0,NaN,1,1
1,ad6561e2d8,c4cd2780ab,2018-11-23 13:42:50,5.9,0.0,0.0,0.0,593.92,658cf09cd4,4752aa4b20,1.0,55.0,NaN,1,1
2,ad6561e2d8,c4cd2780ab,2018-11-23 13:42:50,5.9,0.0,0.0,0.0,593.92,658cf09cd4,3d7742b802,1.0,100.0,NaN,1,1
3,ad6561e2d8,c4cd2780ab,2018-11-23 13:42:50,5.9,0.0,0.0,0.0,593.92,658cf09cd4,c89a550a15,1.0,50.0,NaN,1,1
4,ad6561e2d8,c4cd2780ab,2018-11-23 13:42:50,5.9,0.0,0.0,0.0,593.92,658cf09cd4,5c44498721,1.0,25.0,NaN,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15998947,1430184499,c55462da76,2019-03-15 16:32:10,2.6,0.0,0.0,0.0,526.39,ebf9499b63,5d125fa59a,1.0,120.0,NaN,0,1
15998948,1430184499,c55462da76,2019-03-15 16:32:10,2.6,0.0,0.0,0.0,526.39,ebf9499b63,67e5809518,1.0,32.0,NaN,0,1
15998949,1430184499,c55462da76,2019-03-15 16:32:10,2.6,0.0,0.0,0.0,526.39,ebf9499b63,a449f77e31,1.0,33.0,NaN,0,1
15998950,1430184499,c55462da76,2019-03-15 16:32:10,2.6,0.0,0.0,0.0,526.39,ebf9499b63,29de99801b,1.0,33.0,NaN,0,1


In [109]:
train_df_purchases = train_df.merge(purchases_features, how = 'left', on = 'client_id')
train_df_purchases


test_df_purchases = test_df.merge(purchases_test_features, how = 'left', on = 'client_id')
test_df_purchases

,client_id,first_issue_date,first_redeem_date,age,gender,first_issue_date_d,first_issue_time,first_redeem_time,issue_redeem_delay,transaction_id,product_id
0,a9a604ed6e,2018-09-13 17:36:41,NaN,36,F,2018-09-13,1536860201,NaN,NaN,5,41
1,ebd7360016,2017-07-03 17:12:12,2017-09-01 16:15:12,63,F,2017-07-03,1499101932,1.504283e+09,5180580.0,31,68
2,908cd9b8e8,2018-03-30 17:01:20,2018-07-13 17:05:52,49,F,2018-03-30,1522429280,1.531502e+09,9072272.0,17,75
3,dceb8ce861,2017-10-06 17:06:42,2018-08-11 18:43:58,46,U,2017-10-06,1507309602,1.534013e+09,26703436.0,6,93
4,f4f0ac6b06,2018-06-27 21:39:41,2019-02-15 20:14:37,119,U,2018-06-27,1530135581,1.550262e+09,20126096.0,12,91
...,...,...,...,...,...,...,...,...,...,...,...
60007,4762fb04c5,2018-05-31 16:03:28,2018-12-26 14:03:30,26,M,2018-05-31,1527782608,1.545833e+09,18050402.0,30,127
60008,533d1516e7,2017-09-29 08:27:16,2018-01-07 17:36:31,37,M,2017-09-29,1506673636,1.515347e+09,8672955.0,10,14
60009,c93b21a707,2018-10-03 21:02:26,2019-03-30 17:40:42,42,U,2018-10-03,1538600546,1.553968e+09,15367096.0,34,166
60010,3fe0668de5,2018-12-25 18:27:14,2019-02-02 10:30:53,33,U,2018-12-25,1545762434,1.549103e+09,3341019.0,14,40


In [110]:
test_df_purchases.drop('client_id', axis = 1, inplace = True)

In [111]:
test_df_purchases

,first_issue_date,first_redeem_date,age,gender,first_issue_date_d,first_issue_time,first_redeem_time,issue_redeem_delay,transaction_id,product_id
0,2018-09-13 17:36:41,NaN,36,F,2018-09-13,1536860201,NaN,NaN,5,41
1,2017-07-03 17:12:12,2017-09-01 16:15:12,63,F,2017-07-03,1499101932,1.504283e+09,5180580.0,31,68
2,2018-03-30 17:01:20,2018-07-13 17:05:52,49,F,2018-03-30,1522429280,1.531502e+09,9072272.0,17,75
3,2017-10-06 17:06:42,2018-08-11 18:43:58,46,U,2017-10-06,1507309602,1.534013e+09,26703436.0,6,93
4,2018-06-27 21:39:41,2019-02-15 20:14:37,119,U,2018-06-27,1530135581,1.550262e+09,20126096.0,12,91
...,...,...,...,...,...,...,...,...,...,...
60007,2018-05-31 16:03:28,2018-12-26 14:03:30,26,M,2018-05-31,1527782608,1.545833e+09,18050402.0,30,127
60008,2017-09-29 08:27:16,2018-01-07 17:36:31,37,M,2017-09-29,1506673636,1.515347e+09,8672955.0,10,14
60009,2018-10-03 21:02:26,2019-03-30 17:40:42,42,U,2018-10-03,1538600546,1.553968e+09,15367096.0,34,166
60010,2018-12-25 18:27:14,2019-02-02 10:30:53,33,U,2018-12-25,1545762434,1.549103e+09,3341019.0,14,40


In [112]:
selected_features = ['age','gender','first_issue_time','first_redeem_time','issue_redeem_delay',
                     'transaction_id','product_id', 'treatment_flg']

train_df_purchases[selected_features]
y = train_df_purchases['purchased']


In [113]:
y = train_df_purchases['purchased']


In [114]:
y

0         1
1         1
2         1
3         0
4         1
         ..
140022    1
140023    1
140024    1
140025    1
140026    1
Name: purchased, Length: 140027, dtype: int64

In [115]:
X_train, X_val, y_train, y_val = train_test_split(train_df_purchases[selected_features],y,
                                                  test_size=0.3, random_state=123)
X_train

,age,gender,first_issue_time,first_redeem_time,issue_redeem_delay,transaction_id,product_id,treatment_flg
35393,18,M,1503484834,1.558094e+09,54608948.0,12,40,1
63384,32,M,1496267338,1.498257e+09,1989456.0,41,188,1
48777,78,U,1503919951,1.505240e+09,1320149.0,4,9,1
91057,64,F,1513442311,1.518542e+09,5099581.0,109,183,1
33732,59,F,1491902582,1.529940e+09,38037917.0,18,57,1
...,...,...,...,...,...,...,...,...
129130,41,U,1501271422,1.502131e+09,859616.0,21,101,1
119906,56,U,1501262081,1.520025e+09,18763228.0,6,136,1
17730,33,U,1499430832,1.530559e+09,31128098.0,24,94,0
28030,61,U,1496052246,1.513005e+09,16952991.0,23,75,1


In [116]:
treatment_train = X_train['treatment_flg']
treatment_val = X_val['treatment_flg']

X_train = X_train.drop(columns = ['treatment_flg'])
X_val = X_val.drop(columns = ['treatment_flg'])

In [117]:
y_train

35393     1
63384     1
48777     0
91057     1
33732     0
         ..
129130    1
119906    1
17730     0
28030     0
15725     1
Name: purchased, Length: 98018, dtype: int64

In [118]:
# estimator = CatBoostClassifier(verbose=100, 
#                                cat_features=['gender', 'age'],
#                                random_state=42,
#                                thread_count=8)

# sm = SoloModel(estimator)
# sm = sm.fit(X_train, y_train, treatment_train)

In [119]:
ct = ClassTransformation(CatBoostClassifier(iterations=20, thread_count=2, random_state=42, silent=True))
ct = ct.fit(X_train, y_train, treatment_train, estimator_fit_params={'cat_features': ['gender']})

uplift_ct = ct.predict(X_val)

In [120]:
# uplift_sm = sm.predict(X_val)
sm_score =  qini_auc_score(y_true=y_val, uplift=uplift_ct, treatment=treatment_val)
sm_score

0.02079443067867077

In [121]:
uplift_ct

array([0.03628692, 0.02158138, 0.04110932, ..., 0.10285696, 0.0276082 ,
       0.04011326])

In [122]:
selected_features_for_predict = ['age','gender','first_issue_time','first_redeem_time','issue_redeem_delay',
                     'transaction_id','product_id']
uplift = ct.predict(test_df_purchases[selected_features_for_predict])
uplift

array([-0.00029795,  0.04875072,  0.05664544, ..., -0.12120412,
        0.00672818, -0.06088788])

In [123]:
test['pred'] = uplift
test

,client_id,pred
0,a9a604ed6e,-0.000298
1,ebd7360016,0.048751
2,908cd9b8e8,0.056645
3,dceb8ce861,0.032171
4,f4f0ac6b06,0.063839
...,...,...
60007,4762fb04c5,0.008156
60008,533d1516e7,0.027466
60009,c93b21a707,-0.121204
60010,3fe0668de5,0.006728


In [124]:
test.to_csv('subimission.csv',index=False)

In [125]:
# uplift_sm[uplift_sm < 0]

array([-0.0059649 , -0.02104507, -0.01829557, ..., -0.03831237,
       -0.02197326, -0.00718811])

In [126]:
# train_df_purchases[selected_features]

,age,gender,first_issue_time,first_redeem_time,issue_redeem_delay,transaction_id,product_id,treatment_flg
0,50,F,1512322431,1.527102e+09,14779510.0,40,135,1
1,24,F,1510331629,1.519326e+09,8994516.0,2,14,1
2,36,U,1509657465,1.537045e+09,27387871.0,33,117,1
3,79,F,1526466080,1.527713e+09,1247284.0,13,38,0
4,34,F,1500320188,1.513332e+09,13012137.0,30,67,1
...,...,...,...,...,...,...,...,...
140022,72,U,1540991437,1.554559e+09,13567144.0,4,53,1
140023,35,U,1517752292,1.520877e+09,3124210.0,53,130,0
140024,50,F,1519380153,1.559324e+09,39944103.0,7,33,0
140025,55,U,1498935933,1.508353e+09,9416669.0,17,40,1


In [127]:
# selected_features_for_predict = ['age','gender','first_issue_time','first_redeem_time','issue_redeem_delay',
#                      'transaction_id','product_id']
# uplift_sm = sm.predict(test_df_purchases[selected_features_for_predict])
# uplift_sm

array([ 0.00969749,  0.02798543,  0.03661914, ..., -0.00718811,
        0.03838962,  0.03580835])

In [128]:
# test['pred'] = uplift_sm
# test

,client_id,pred
0,a9a604ed6e,0.009697
1,ebd7360016,0.027985
2,908cd9b8e8,0.036619
3,dceb8ce861,0.019031
4,f4f0ac6b06,0.043224
...,...,...
60007,4762fb04c5,0.016236
60008,533d1516e7,0.053641
60009,c93b21a707,-0.007188
60010,3fe0668de5,0.038390


In [129]:
# sm_score

0.02079443067867077

In [130]:
# test.to_csv('subimission.csv',index=False)

In [131]:
#0.0082375 - ['gender']
#0.0069618 - ['gender', age]
#0.0096642 - thread_count=4
#0.0111866 - thread_count=8